In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:90%;}
div.CodeMirror {font-family:Consolas; font-size:20pt;}
div.output {font-size:18pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:19pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
span.toc-item-num{display:none;}
div.text_cell_render ul li{font-size:16pt;padding:5px;}
div.CodeMirror-lines > div {padding-left:10px;}
table.dataframe{font-size:19px;}
</style>
"""))

In [ ]:
import numpy as np
import pandas as pd # read_csv, get_dumies(원핫인코딩)
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split # 훈련셋과 시험셋 나누기
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.callbacks import EarlyStopping, Callback
from tensorflow.keras import metrics # metrics = ['accuracy']
import matplotlib.pyplot as plt

- Red Wine 품질 등급 예측

```
1. 데이터 셋 확보 & 전처리
    독립변수, 종속변수 분리 -> 독립변수 스케일조정(StandardScaler), 종속변수 원핫인코딩(get_dummies), 
     훈련셋과 테스트셋 분리(train_test_split)
2. 모델 구성(입력11, 출력6) layer층 4
3. 모델 학습 과정 설정(다중분류에 맞는 설정)
4. 학습시키기 (callbacks이용)
5. 모델 평가 - 그래프, 평가(테스트셋), 교차표(crosstab)
6. 모델 저장 및 모델사용하기
```

In [ ]:
redwine = pd.read_csv('./data/winequality-red.csv', sep=';')
redwine.quality.unique()

In [ ]:
redwine.head()